# Week 01 — Logprobs fix & baseline relaunch

**Goal**: validate token logprobs and relaunch baseline.

In [ ]:
!python3 nb.py --bases experiment_archive results --out reports/week01

Figures will be saved under `figures/week01`.